In [1]:
# 4.2.1 DATA PREPARATION
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, davies_bouldin_score
from sklearn.decomposition import PCA

# Load dataset
df = pd.read_excel('DatasetWestJava.xlsx', sheet_name='Sheet1')

# Atur header yang benar
df.columns = df.iloc[0]
df = df[1:].reset_index(drop=True)

# Pilih dan rename kolom penting
df_selected = df[['nama_kabupaten_kota', 'jumlah_penderita_dm',
                  'jumlah_ warga_terdaftar_bpjs', 'jumlah_puskesmas', 'tahun']].copy()
df_selected.columns = ['kabupaten_kota', 'penderita_dm', 'bpjs', 'puskesmas', 'tahun']

# 4.2.2 DATA CLEANING & PREPROCESSING
# Deteksi dan hapus duplikat
print("Jumlah data duplikat:", df_selected.duplicated().sum())
df_selected = df_selected.drop_duplicates()

# Konversi ke numerik dan deteksi missing values
for col in ['penderita_dm', 'bpjs', 'puskesmas']:
    df_selected[col] = pd.to_numeric(df_selected[col], errors='coerce')

print("\nJumlah missing value per kolom:")
print(df_selected.isnull().sum())

# Drop missing value
df_clean = df_selected.dropna().reset_index(drop=True)

# Normalisasi fitur
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df_clean[['penderita_dm', 'bpjs', 'puskesmas']])

# 4.2.3 SPLITTING DATA — Tidak dilakukan (Unsupervised Learning)

# 4.2.4 MODEL DEVELOPMENT - ELBOW METHOD & SILHOUETTE SCORE
inertia = []
silhouette_scores = []
K_range = range(2, 10)

for k in K_range:
    km = KMeans(n_clusters=k, random_state=42)
    km.fit(X_scaled)
    inertia.append(km.inertia_)
    silhouette_scores.append(silhouette_score(X_scaled, km.labels_))

# Visualisasi Elbow dan Silhouette Score
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(K_range, inertia, marker='o')
plt.title('Elbow Method')
plt.xlabel('Jumlah Cluster')
plt.ylabel('Inertia')
plt.grid(True)

plt.subplot(1, 2, 2)
plt.plot(K_range, silhouette_scores, marker='o', color='green')
plt.title('Silhouette Score per Jumlah Cluster')
plt.xlabel('Jumlah Cluster')
plt.ylabel('Silhouette Score')
plt.grid(True)

plt.tight_layout()
plt.show()

# Cetak nilai Silhouette Score
print("\n📊 Silhouette Score per jumlah cluster:")
for k, score in zip(K_range, silhouette_scores):
    print(f'Jumlah Cluster = {k} → Silhouette Score = {score:.4f}')

# Gunakan k=3 berdasarkan hasil Elbow dan Silhouette
kmeans = KMeans(n_clusters=3, random_state=42)
df_clean['cluster'] = kmeans.fit_predict(X_scaled)

# 4.2.5 MATRIX EVALUATION
sil_score = silhouette_score(X_scaled, df_clean['cluster'])
dbi_score = davies_bouldin_score(X_scaled, df_clean['cluster'])

print(f'\n📈 Silhouette Score (k=3): {sil_score:.4f}')
print(f'📉 Davies-Bouldin Index (DBI): {dbi_score:.4f}')

# 4.2.6 PERFORMANCE MODEL EVALUATION
print("\nJumlah anggota tiap cluster:")
print(df_clean['cluster'].value_counts())

# 4.2.7 LABELING CLUSTER
cluster_mean = df_clean.groupby('cluster')[['penderita_dm', 'bpjs', 'puskesmas']].mean()
print("\nRata-rata per cluster:")
print(cluster_mean)

# Label kategori berdasarkan jumlah penderita DM
cluster_order = cluster_mean['penderita_dm'].sort_values(ascending=False).index.tolist()
kategori_map = ['Tertinggi', 'Menengah', 'Rendah']
cluster_label_map = {cluster_id: kategori_map[i] for i, cluster_id in enumerate(cluster_order)}

# Warna visualisasi
cluster_colors = {
    'Tertinggi': '#FF0000',
    'Menengah': '#FFA500',
    'Rendah': '#8B4513'
}

# Tambahkan kategori dan warna
df_clean['kategori'] = df_clean['cluster'].map(cluster_label_map)
df_clean['color'] = df_clean['kategori'].map(cluster_colors)

# 4.2.8 PCA 2D UNTUK VISUALISASI CLUSTER (Dipindah ke atas)
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

df_clean['pca1'] = X_pca[:, 0]
df_clean['pca2'] = X_pca[:, 1]

# Visualisasi PCA
plt.figure(figsize=(6, 4))
for kategori in df_clean['kategori'].unique():
    subset = df_clean[df_clean['kategori'] == kategori]
    plt.scatter(subset['pca1'], subset['pca2'],
                label=kategori, color=subset['color'].iloc[0], alpha=0.7, edgecolors='k')

plt.title('Visualisasi Cluster dengan PCA 2D')
plt.xlabel('PCA 1')
plt.ylabel('PCA 2')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# 9. VISUALISASI PER TAHUN
tahun_list = sorted(df_clean['tahun'].unique())

for tahun in tahun_list:
    df_tahun = df_clean[df_clean['tahun'] == tahun]

    plt.figure(figsize=(14, 6))
    bars = plt.bar(df_tahun['kabupaten_kota'], df_tahun['penderita_dm'],
                   color=df_tahun['color'])

    for bar, label in zip(bars, df_tahun['kategori']):
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width() / 2.0, height + 1000,
                 label, ha='center', va='bottom', fontsize=8, rotation=90)

    plt.xticks(rotation=90)
    plt.title(f'Segmentasi Kabupaten/Kota Tahun {tahun} Berdasarkan Jumlah Penderita DM')
    plt.ylabel('Jumlah Penderita DM')
    plt.tight_layout()
    plt.show()

# 10. TABEL REKAP CLUSTER (TABEL 2)
tabel_cluster = df_clean.groupby(['cluster', 'kategori']).agg(
    jumlah_daerah=('kabupaten_kota', 'count'),
    total_penderita_dm=('penderita_dm', 'sum')
).reset_index()

# Urutkan kategori secara logis
kategori_order = ['Rendah', 'Menengah', 'Tertinggi']
tabel_cluster['kategori'] = pd.Categorical(tabel_cluster['kategori'],
                                           categories=kategori_order, ordered=True)
tabel_cluster = tabel_cluster.sort_values('kategori')

print("\n📊 Tabel 2. Rekap Hasil Cluster:")
print(tabel_cluster)

print("\n📋 Daftar Kabupaten/Kota per Cluster (Per Tahun):")
for tahun in tahun_list:
    df_tahun = df_clean[df_clean['tahun'] == tahun]
    df_tabel = df_tahun[['kabupaten_kota', 'tahun', 'cluster', 'kategori',
                         'penderita_dm', 'bpjs', 'puskesmas']]
    print(f"\nTahun {tahun}:")
    print(df_tabel.sort_values('cluster').to_string(index=False))

print("\n📋 Daftar Keseluruhan Kabupaten/Kota per Cluster:")
df_tabel_all = df_clean[['kabupaten_kota', 'tahun', 'cluster', 'kategori',
                         'penderita_dm', 'bpjs', 'puskesmas']]
print(df_tabel_all.sort_values(['cluster', 'tahun', 'kabupaten_kota']).to_string(index=False))


FileNotFoundError: [Errno 2] No such file or directory: 'Dataset10.xlsx'